# Pyomo Homework 2-PT

In [7]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.install_glpk()

--2023-02-02 02:19:55--  https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6463 (6.3K) [text/plain]
Saving to: ‘helper.py.1’

helper.py.1         100%[===================>]   6.31K  --.-KB/s    in 0s      

2023-02-02 02:19:55 (68.7 MB/s) - ‘helper.py.1’ saved [6463/6463]

IDAES found! No need to install.
Ipopt found! No need to install.
ipopt was successfully installed
k_aug was successfully installed
Installing glpk via apt-get...


In [8]:
## IMPORT LIBRARIES
import pyomo.environ as pyo
import pandas as pd

Special thanks to the Pyomo team for create these excercises as part of their excellent PyomoFest workshop.

## Some Advanced Pyomo Tricks

### Using the decorator notation for rules

Alternative notation for declaring and defining Pyomo components using decorators exists. Starting with the warehouse location problem code below, change the model to use the decorator notation.

In [9]:
# warehouse_location.py: Warehouse location determination problem
model = pyo.ConcreteModel(name="(WL)")

W = ['Harlingen', 'Memphis', 'Ashland']
C = ['NYC', 'LA', 'Chicago', 'Houston']
d = {('Harlingen', 'NYC'): 1956, \
     ('Harlingen', 'LA'): 1606, \
     ('Harlingen', 'Chicago'): 1410, \
     ('Harlingen', 'Houston'): 330, \
     ('Memphis', 'NYC'): 1096, \
     ('Memphis', 'LA'): 1792, \
     ('Memphis', 'Chicago'): 531, \
     ('Memphis', 'Houston'): 567, \
     ('Ashland', 'NYC'): 485, \
     ('Ashland', 'LA'): 2322, \
     ('Ashland', 'Chicago'): 324, \
     ('Ashland', 'Houston'): 1236 }
P = 2

model.x = pyo.Var(W, C, bounds=(0,1))
model.y = pyo.Var(W, within=pyo.Binary)

@model.Objective()
def obj(m):
    return sum(d[w,c]*m.x[w,c] for w in W for c in C)

@model.Constraint(C)
def one_per_cust(m, c):
    return sum(m.x[w,c] for w in W) == 1

# Add your solution here
"""
My own notes:
x_(wc) -> is the percentage of people that will get the product from the warehous
^__ So the above is unknown
d_(wc) -> the cost of delivering from a warehous to customer
What we want to minimize? total_cost => d_(wc)*x_(wc)
Let me understand the constraints?
Can the total fraction be more than 1? -> No, But the total should be equal to 1 
to make sure we have all customer demand!

What is the P? It is the total number of products (another constraint)
What is y? did we select the warehouse or not (0, 1)
"""
@model.Constraint(W, C)
def warehouse_active(m, w, c):
    return m.x[w,c] <= m.y[w]

# Note: This is only split across cells because of a bug in nbpages (notebook/website software).
# There is no other reason to split your code across cells.

In [10]:
# Add your solution here
@model.Constraint()
def num_warehouses(m):
    return sum(m.y[w] for w in W) <= P

pyo.SolverFactory('glpk').solve(model)

model.y.pprint()
model.x.pprint()

y : Size=3, Index=y_index
    Key       : Lower : Value : Upper : Fixed : Stale : Domain
      Ashland :     0 :   1.0 :     1 : False : False : Binary
    Harlingen :     0 :   1.0 :     1 : False : False : Binary
      Memphis :     0 :   0.0 :     1 : False : False : Binary
x : Size=12, Index=x_index
    Key                      : Lower : Value : Upper : Fixed : Stale : Domain
      ('Ashland', 'Chicago') :     0 :   1.0 :     1 : False : False :  Reals
      ('Ashland', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
           ('Ashland', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
          ('Ashland', 'NYC') :     0 :   1.0 :     1 : False : False :  Reals
    ('Harlingen', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
    ('Harlingen', 'Houston') :     0 :   1.0 :     1 : False : False :  Reals
         ('Harlingen', 'LA') :     0 :   1.0 :     1 : False : False :  Reals
        ('Harlingen', 'NYC') :     0 :   0.0 :     1 : False : False : 

### Changing parameter values

A parameter can be specified to be mutable. This tells Pyomo that the value of the parameter may change in the future, and allows the user to change the parameter value and resolve the problem without the need to rebuild the entire model each time. We will use this functionality to find a better solution to the knapsack problem. We would like to find when the wrench becomes valuable enough to be a part of the optimal solution. Create a Pyomo Parameter for the value of the items, make it mutable, and then write a loop that prints the solution for different wrench values.

In [12]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )
# Add your solution here
"""
Here, the value that is going to change is the wrench value,
therefore, we define b as the mutable item.
The important point here is defining the set (here the keys of dict) 
to catch the initial value
"""
model.init = pyo.Set(initialize=A)
model.item_benefit = pyo.Param(model.init, initialize=b, mutable=True)


def obj_rule(m):
    return sum( m.item_benefit[i]*m.x[i] for i in A )
model.obj = pyo.Objective(rule=obj_rule, sense = pyo.maximize )

def weight_rule(m):
    return sum( w[i]*m.x[i] for i in A ) <= W_max
model.weight = pyo.Constraint(rule=weight_rule)

opt = pyo.SolverFactory('glpk')

for wrench_benefit in range(1,11):
    model.item_benefit['wrench'] = wrench_benefit
    result_obj = opt.solve(model)   
    # Add your solution here
    total_weight = sum( w[i]*pyo.value(model.x[i]) for i in A )
    print("============================================================")
    print("====================Iteration Number: %d=====================" %wrench_benefit)
    print("============================================================")
    print("The total weight of items aquired in the optimal solution: ", round(total_weight, 2))
    print('----------------------------------------------')  
    print('%12s %12s %12s' % ('Item', 'Selected', 'Value'))
    print('----------------------------------------------')  
    for i in A:
      print('%12s %12s %12s' % (i, pyo.value(model.x[i]), pyo.value(model.item_benefit[i])*pyo.value(model.x[i])))
      print('----------------------------------------------')  
    print("\n")

====================Iteration Number: 1=====================
The total weight of items aquired in the optimal solution:  12.0
----------------------------------------------
        Item     Selected        Value
----------------------------------------------
      hammer          1.0          8.0
----------------------------------------------
      wrench          0.0          0.0
----------------------------------------------
 screwdriver          1.0          6.0
----------------------------------------------
       towel          1.0         11.0
----------------------------------------------


====================Iteration Number: 2=====================
The total weight of items aquired in the optimal solution:  12.0
----------------------------------------------
        Item     Selected        Value
----------------------------------------------
      hammer          1.0          8.0
----------------------------------------------
      wrench          0.0          0.0
-----------

### Integer cuts

Often, it can be important to find not only the "best" solution, but a number of solutions that are equally optimal, or close to optimal. For discrete optimization problems, this can be done using something known as an integer cut. Consider again the knapsack problem where the choice of which items to select is a discrete variable $x_i \forall i \in A$. Let $x_i^*$ be a particular set of $x$ values we want to remove from the feasible solution space. We define an integer cut using two sets. The first set $S_0$ contains the indices for those variables whose current solution is 0, and the second set $S_1$ consists of indices for those variables whose current solution is 1. Given these two sets, an integer cut constraint that would prevent such a solution from appearing again is defined by,

$\sum_{i \in S_0}x[i] + \sum_{i \in S_1}(1-x_i) \geq 1$

Write a loop that solves the problem 5 times, adding an integer cut to remove the previous solution, and printing the value of the objective function and the solution at each iteration of the loop.

In [13]:
from pyomo.environ import *

A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = pyo.ConcreteModel()
model.x = pyo.Var( A, within=pyo.Binary )

def obj_rule(m):
    return sum( b[i]*m.x[i] for i in A )
model.obj = pyo.Objective(rule=obj_rule, sense = maximize )

def weight_con_rule(m):
    return sum( w[i]*m.x[i] for i in A ) <= W_max
model.weight_con = Constraint(rule=weight_con_rule)

opt = pyo.SolverFactory('glpk')


# create the ConstraintList to hold the integer cuts
model.int_cuts = ConstraintList()

# Add your solution here
"""
Based on the Pyomo Doc, for defining the integer cut, we need to use the 
ConstraintList(), then based on the model constraint definition we define
where to add each set off integer cut, here, for model variables that their 
"value" are equal to zero we define the first set formulation (if condition), 
and for other condition we define the second set (elif or else condition).
"""
result_obj = opt.solve(model) 
for i in range(5):
  expr = 0
  for j in A:
    if pyo.value(model.x[j]) == 0:
        expr += model.x[j]
    elif pyo.value(model.x[j]) == 1:
        expr += (1 - model.x[j])
  model.int_cuts.add( expr >= 1 )
  results = opt.solve(model)
  print("============================================================")
  print("====================Iteration Number: %d=====================" %(i+1))
  print("============================================================")
  """
  For printing the required output we need to define the item, selected or not,
  and the objective value which is the sum of the benefit
  """
  print("The objective function value for the optimal solution: ", 
        round(sum( b[i]*pyo.value(model.x[i]) for i in A ), 2))
  print("============================================================")
  print("The total weight: ", 
        round(sum( w[i]*pyo.value(model.x[i]) for i in A ), 2))
  print("============================================================")
  print('--------------------------------------------------------')  
  print('%12s %12s %12s' % ('Item', 'Selected', 'Value'))
  print('--------------------------------------------------------')  
  for i in A:
    print('%12s %12s %12s' % (i, pyo.value(model.x[i]), b[i]*pyo.value(model.x[i])))
    print('--------------------------------------------------------')  
  print("\n")

====================Iteration Number: 1=====================
The objective function value for the optimal solution:  20.0
The total weight:  14.0
--------------------------------------------------------
        Item     Selected        Value
--------------------------------------------------------
      hammer          0.0          0.0
--------------------------------------------------------
      wrench          1.0          3.0
--------------------------------------------------------
 screwdriver          1.0          6.0
--------------------------------------------------------
       towel          1.0         11.0
--------------------------------------------------------


====================Iteration Number: 2=====================
The objective function value for the optimal solution:  19.0
The total weight:  8.0
--------------------------------------------------------
        Item     Selected        Value
--------------------------------------------------------
      hammer     

### Putting it all together: Lot sizing example (Hart et al., 2017)

We will now write a complete model from scratch using a well-known multi-period optimization problem for optimal lot-sizing adapted from Hagen et al. (2001) shown below.

$\min \sum_{t \in T}c_ty_t+h_t^+I_t^+ +h_t^-I_t^-$

s.t. $I_t=I_{t-1}+X_t-d_t, \forall t \in T$

$I_t=I_t^+-I_t^-, \forall t \in T$

$X_t \leq Py_t, \forall t \in T$

$X_t, I_t^+, I_t^- \geq 0, \forall t \in T$

$y_t \in \{0,1\}, \forall t \in T$

Our goal is to finnd the optimal production $X_t$ given known demands $d_t$, fixed cost $c_t$ associated with active production in a particular time period, an inventory holding cost $h_t^+$ and a shortage cost $h_t^-$ (cost of keeping a backlog) of orders. The variable $y_t$ (binary) determines if we produce in time $t$ or not, and $I_t^+$ represents inventory that we are storing across time period $t$, while $I_t^-$ represents the magnitude of the backlog. Note that $X_t \leq Py_t$ is a constraint that only allows production in time period $t$ if the indicator variable $y_t$=1.

Write a Pyomo model for this problem and solve it using glpk using the data provided below.

|Parameter|Description|Value|
|---|---|---|
|$c$|fixed cost of production|4.6|
|$I_0^+$|initial value of positive inventory|5.0|
|$I_0^-$|initial value of backlogged orders|0.0|
|$h^+$|cost (per unit) of holding inventory|0.7|
|$h^-$|shortage cost (per unit)|1.2|
|$P$|maximum production amoung (big-M value)|5|
|$d$|demand|[5,7,6.2,3.1,1,7]|

**Reference**: Hart, W. E., Laird, C. D., Watson, J. P., Woodruff, D. L., Hackebeil, G. A., Nicholson, B. L., and Siirola, J. D. Pyomo: Optimization Modeling in Python (Second Edition), Vol (67), Springer Verlag, 2017.

In [14]:
model = pyo.ConcreteModel()
model.T = RangeSet(5)    # time periods

i0 = 5.0           # initial inventory
c = 4.6            # setup cost
h_pos = 0.7        # inventory holding cost
h_neg = 1.2        # shortage cost
P = 5.0            # maximum production amount

# demand during period t
d = {1: 5.0, 2:7.0, 3:6.2, 4:3.1, 5:1.7}

# Add your solution here
"""
My own notes:
1) The model is dependent on the time period, so, we need a domain for that
which here was defined by the Rangeset 
2) Define each model variables:
Based on the objective function: 
y_t-> Binary value (we produce or not)-> Time dependent(index),Binary (domain)
I-> Inventory-> Time dependent (index), Nospecific domain
I+ and I- are variables used to define the inventory: important point about them
is that 'both of them are positive'-> Time dependent(index)
Based on the constraints:
x_t-> Optimal production-> Time dependent (index), Nonnegative (domain, real)
3) The other parts is all about the implementing the provided formula
"""
# ==========================> Defining model variable
model.y_t = Var(model.T, domain=Binary)
model.I = Var(model.T)
model.I_pos = Var(model.T, domain=NonNegativeReals)
model.I_neg = Var(model.T, domain=NonNegativeReals)
model.x_t = Var(model.T, domain=NonNegativeReals)
# ==========================> Defining objective function
def obj_rule(m):
  return sum(c*m.y_t[t] + h_pos*m.I_pos[t] + h_neg*m.I_neg[t] for t in m.T)
model.obj = Objective(rule=obj_rule)
# ==========================> Defining the model constraints
# Equality Constraints
def constraintA(m, t):
  # As it is depend on previous value we need seperate the formulation 
  # for the first value of the t (t0) and then the other value of the t (dependants)
  if t == m.T.first():
    return m.I[t] == i0 + m.x_t[t] - d[t]
  else:
    return m.I[t] == m.I[t-1] + m.x_t[t] - d[t]
model.conA = Constraint(model.T, rule=constraintA)
#
def ConstraintB(m, t):
  # This based on the formulation
  return m.I[t] == m.I_pos[t] - m.I_neg[t]
model.conB = Constraint(model.T, rule=ConstraintB)
#
# Inequality Constraints
def ConstraintC(m,t):
  # This is based on the formulation
  return m.x_t[t] <= P*m.y_t[t]
model.conC = Constraint(model.T, rule=ConstraintC)
#
# solve the problem
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# print the results
for t in model.T:
    print('Period: {0}, Prod. Amount: {1}'.format(t, value(model.x_t[t]))) 

Period: 1, Prod. Amount: 3.0
Period: 2, Prod. Amount: 5.0
Period: 3, Prod. Amount: 5.0
Period: 4, Prod. Amount: 5.0
Period: 5, Prod. Amount: 0.0


## Nonlinear programs: initial and problem formulation are very important!

### Alternative initialization

Effective initialization can be critical for solving nonlinear problems, since they can have several local solutions and numerical diffculties. Solve the [Rosenbrock problem](https://en.wikipedia.org/wiki/Rosenbrock_function) using different initial values for the x variables. Write a loop that varies the initial value from 2.0 to 6.0, solves the problem, and prints the solution for each iteration of the loop.

In [15]:
model = ConcreteModel()
model.x = Var()
model.y = Var()

def rosenbrock(m):
    return (1.0-m.x)**2 + 10000.0*(m.y - m.x**2)**2
model.obj = pyo.Objective(rule=rosenbrock, sense=minimize)


solver = pyo.SolverFactory('ipopt')

### Add your solution here
print('=======================================================================') 
print('%12s | %12s | %18s | %12s' % ('x_init', 'y_init', 'x_soln', 'y_soln'))
print('=======================================================================')
# I also define a constant intial value for the model variable y:
# In addition after running the solution values for me were somehow wierd
y_init = 3
for init in range(2, 7): 
  """
  Based on the Pyomo Doc for variable:
  Instead of the initialize option, initialization is sometimes done with a 
  Python assignment statement
  I use this idea to change the initialize value
  """
  model.x = init
  model.y = y_init
  solver = pyo.SolverFactory('ipopt')
  solver.solve(model)
  print('%12s | %12s | %12s | %12s' % (init,y_init, pyo.value(model.x), pyo.value(model.y)))
print('=======================================================================')


      x_init |       y_init |             x_soln |       y_soln
           2 |            3 | 1.0000000078869047 | 1.0000000157495166
           3 |            3 | 1.0000000000680371 | 1.0000000001359306
           4 |            3 | 1.0000000112221112 | 1.000000022117188
           5 |            3 | 1.0000000046401478 | 1.0000000091696122
           6 |            3 | 1.000000014678325 | 1.0000000278447572


As elaborated here, the [Rosenbrock problem](https://en.wikipedia.org/wiki/Rosenbrock_function) is a classic "hard" test case for optimization algorithms. Your results may surprise you (and show the effectiveness of Pyomo and Ipopt!).

### Evaluation errors

Consider the following problem with **initial values** $x$=5, $y$=5.

$\min_{x,y} f(x,y)=(x-1.01)^2+y^2$

s.t. $y=\sqrt{x-1.0}$


1. Formulate this Pyomo model and solve using IPOPT. You should get a list of errors from the solver. Add the IPOPT solver option solver.options['halt_on_ampl_error']='yes' to find the problem. *Hint*: the error output might be ordered strangely, look this up in the console output. What did you discover? How might you fix this?

In [16]:
# Add your solution here
"""
The steps to create the optimization problem:
"""
model = pyo.ConcreteModel()
model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

@model.Objective()
def obj_rule(m):
  return (m.x-1.01)**2+m.y**2

@model.Constraint()
def con_rule(m):
  return m.y == sqrt(m.x - 1)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        c

ERROR:pyomo.opt:Solver (ipopt) returned non-zero return code (1)
ERROR:pyomo.opt:See the solver log above for diagnostic information.


ApplicationError: ignored

**Question Answers**

The error is as follows:

Error evaluating "var =" definition -1: can't evaluate sqrt(-0.752432).

As it can be seen in the problem definition, we have $y = \sqrt{x - 1}$ which is not defined for the $ x < 1$. Hence, in the problem definiton we need to define a boundary condition for the variable $x$ as $x \geq 1$.
This shows the effect of the boundary value for the problem to help the algorithm to converge and be solvable. 

2. Add bounds $x \geq 1$ to fix this problem and resolve. Comment on the number of iterations and the quality of solution. Note, the problem still occurs because $x \geq 1$ is not enforced exactly, and small numerical values still cause the error.

In [17]:
from math import inf
from pyomo.core.base.piecewise import Bound
# Add your solution here
model = pyo.ConcreteModel()
model.x = pyo.Var(initialize=5.0, bounds=(1, inf))
model.y = pyo.Var(initialize=5.0)

@model.Objective()
def obj_rule(m):
  return (m.x-1.01)**2+m.y**2

@model.Constraint()
def con_rule(m):
  return m.y == sqrt(m.x - 1)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)


Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        c

ERROR:pyomo.opt:Solver (ipopt) returned non-zero return code (1)
ERROR:pyomo.opt:See the solver log above for diagnostic information.


ApplicationError: ignored

**Discussion**

Again we face the same erros as follow:
Error evaluating "var =" definition -1: can't evaluate sqrt(-9.90312e-09).

As it can be seen for the really small value again we have the constraint problem. To avoid such problem, we use the following trick. Imagin we want to solve some problems that are not defined around zero, for instance $log(0)$. But we need to define a value of $x=0$ due to the problem type. In order to avoid the error, we can define a value of $x~0$ like $x=0.000001$. This helps us to manage the error.

For this specific problem, the boundary can be defined as:

Boundaries = $[1.001, ∞)$ to avoid numerical problem.

Number of iterations:
Here, by defining the boundary the number of iterations increased which shows that the algorithm starts to slove the problem. However, it does not converge to feasible point due to the numerical problem.

Quality of the solution:
Does the algorithm find the solution? The answer is No, as around the lower boundary the algorithm face the numerical problem.

3. Think about other solutions for this problem and attempt to implement one of these solutions. *Hint*: $x \geq 1.001$.

In [18]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0, bounds=(1.001,None))
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.y == sqrt(m.x - 1.0)
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        c

### Alternative formulations

Consider the following problem with initial values $x$=5, $y$=5.

$\min_{x,y} f(x,y)=(x-1.01)^2+y^2$

s.t. $\frac{x-1}{y}=1$

Note, the solution to this problem is $x$=1.005 and $y$=0.005. There are several ways that the problem above can be reformulated. Some examples are shown below. Which ones do you expect to be better? Why?

1. $\min_{x,y} f(x,y)=(x-1.01)^2+y^2$

s.t. $\frac{x-1}{y}=1$

2. $\min_{x,y} f(x,y)=(x-1.01)^2+y^2$

s.t. $\frac{x}{y+1}=1$

3. $\min_{x,y} f(x,y)=(x-1.01)^2+y^2$

s.t. $y=x-1$

Implement Pyomo models for each formulation and solve with IPOPT.

#### Formulation 1

In [19]:
# Add your solution here
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return (m.x - 1) / m.y == 1
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        c

#### Formulation 2

In [20]:
# Add your solution here
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.x / (m.y + 1) == 1
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        c

#### Formulation 3

In [21]:
# Add your solution here
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.y == m.x -1
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        c

Note the number of iterations and quality of solutions. What can you learn about the problem formulation from these examples?

**Discussion**

The solution of the problem for each formulation is as follows:

|Eq|X|Y|Abs Error X|Abs Error Y|Iteration|
|---|---|---|---|---|---|
|$1$|1.00499|0.004999|~0|~0|93|
|$2$|1.005|0.005000|~0|~0|9|
|$3$|1.005|0.004999|~0|~0|1|

Number of iterations:
As it can be seen in the formulation 1 after 93 iteration, the algoritm converges to the feasible point which is the large number for such problem.

In the second formula by modifying the denominator (somehow like adding the effect of boundary), the algorithm converges to the feasible point, after 9 iterations.

In the third formula, by removing the root effect again the solution converges to the feasible point but with the less number of iterations (one iteration).

Quality of solution:
The algorithm perform correctly on all three problems as all of them converge to the feasible point.

This problem shows that the constraint formulation can directly affect the efficiency of the algorithm.


Bounds and initialization can be very helpful when solving nonlinear optimization problems. Resolve the original problem below, but add bounds, $y \geq 0$. Note the number of iterations and quality of solution, and compare with what you found for Formulation 1.

In [22]:
# Add your solution here
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0, bounds=(0,None))

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return (m.x - 1) / m.y == 1
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

Ipopt 3.13.2: halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        c

**Discussion**

|Type|X|Y|Iteration|
|---|---|---|---|
|$Without Boundary (1)$|1.00499|0.00499|93|
|$With Boundary (last)$|1.0050001257911454|0.00500012579114542|38|

Quality of the solution: In both cases the algoritm converges to the feasible point with error around the zero.

Number of iteration:
By defining the boundary for the y, the number of iterations decreases significantly. But this is not neccessarily the best approach. Because, the approach using altrenative formulation can imrove the algorithm efficiency more.


### Reactor design problem (Hart et al., 2017; Bequette, 2003)

Here we will consider a chemical reactor designed to produce product B from reactant A using a reaction scheme known as the Van de Vusse reaction:

$A \overset{k_1}{\rightarrow} B \overset{k_2}{\rightarrow} C$

$2A \overset{k_3}{\rightarrow} D$

Under appropriate assumptions, $F$ is the volumetric flowrate through the tank. The concentation of component A in the feed is $c_{Af}$, and the concentrations in the reactor are equivalent to the concentrations of each component 
flowing out of the reactor, given by $c_A$, $c_B$, $c_C$, and $c_D$.

If the reactor is too small, we will not produce sufficient quantity of B, and if the reactor is too large, much of B will be further reacted to form the undesired product C. Therefore, our goal is to solve for the reactor volume that maximizes the outlet concentration for product B.

The steady-state mole balances for each of the four components are given by

$0=\frac{F}{V}c_{Af}-\frac{F}{V}c_A-k_1C_A-2k_3c^2_A$

$0=-\frac{F}{V}c_{B}+k_1C_A-k_2c_B$

$0=-\frac{F}{V}c_{C}+k_2c_B$

$0=-\frac{F}{V}c_{D}+k_3c^2_A$

The known parameters for the system are:

$c_{Af}=10\frac{\mathrm{gmol}}{\mathrm{m}^3}$

$k_1=\frac{5}{6}\mathrm{min}^{-1}$

$k_2=\frac{5}{3}\mathrm{min}^{-1}$

$k_3=\frac{1}{6000}\frac{\mathrm{m}^3}{\mathrm{gmol}~\mathrm{min}^{-1}}$

Formulate and solve this optimization problem using Pyomo. Since the volumetric flowrate $F$ always appears as the numerator over the reactor volume $V$, it is common to consider this ratio as a single variable, called the space-velocity $SV$.

**References**:
Hart, W. E., Laird, C. D., Watson, J. P., Woodruff, D. L., Hackebeil, G. A., Nicholson, B. L., and Siirola, J. D. Pyomo: Optimization Modeling in Python (Second Edition), Vol (67), Springer Verlag, 2017.

B.W. Bequette. Process control: modeling, design, and simulation. Prentice Hall, 2003.

In [23]:
# Add your solution here
# ==========================> STEP1:  Creating a concrete Pyomo model
model = pyo.ConcreteModel()
# ==========================> STEP2:  Defining model variable
"""
Here , we define the unknown parameters of the model, which at the
first glance are cA, cB, cC, and cD.
There is another parameter named (F/V or SV). I think this parameter can be 
considered as an unknown (model variable), or it can be defined as the model 
parameter that change for the user variables.
I take it here as an unknown.
"""
# !!! the values cannot be negative
model.CA = pyo.Var(initialize = 2000, within=PositiveReals) #gmol/m3
model.CB = pyo.Var(initialize = 2000, within=PositiveReals) #gmol/m3
model.CC = pyo.Var(initialize = 2000, within=PositiveReals) #gmol/m3
model.CD = pyo.Var(initialize = 2000, within=PositiveReals) #gmol/m3
model.SV = pyo.Var(initialize = 1, within=PositiveReals)    #rate
# ==========================> STEP3:  Parameters (Constants / Data)
"""
Defining the parameter of the model that are known using the Pyo.Param or directly
define the value of them.
For our case, they are:
CAF, k1, k2, k3
"""
CAF = 10000 #gmol/m3
k1 = 5/6 #min^-1
k2 = 5/3 #min^-1
k3 = 1/6000 #m3/(gmol*min^-1)
# ==========================> STEP4:  Defining the model objective
"""
What is our goal? Maximizes the outlet concentration for product B (which mean
maximizing CB)
"""
model.obj = pyo.Objective(expr= model.CB, sense = maximize)
# ==========================> STEP5:  Defining the model constraints
"""
We have four equality constraints that should be defined
"""
def balanceA(model):
  return (0 == model.SV * CAF \
              - model.SV * model.CA - k1 * model.CA \
              - 2.0 * k3 * model.CA ** 2.0)
model.conA = pyo.Constraint(rule=balanceA)
#
def balanceB(model):
  return (0  == -model.SV * model.CB \
                + k1 * model.CA - k2 * model.CB)
model.conB = pyo.Constraint(rule=balanceB)
#
def balanceC(model):
  return (0 == -model.SV * model.CC \
               + k2 * model.CB)
model.conC = pyo.Constraint(rule=balanceC)
#
def balanceD(model):
  return (0 == -model.SV * model.CD \
               + k3 * model.CA ** 2.0)
model.conD = pyo.Constraint(rule=balanceD)
# ==========================> STEP6:  Solving the model
solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model)
print("=======================================================")
print("{:<6} {:<10} {:<8} {:<9} {:<10} {:<10}".format("SV", "CAF", "Ca", "Cb", "Cc", "Cd"))
print("=======================================================")
print(round(model.SV.value, 2), "|", 
      CAF,  "|",
      round(model.CA.value, 2),  "|",
      round(model.CB.value, 2),  "|",
      round(model.CC.value, 2),  "|",
      round(model.CD.value, 2))
print("=======================================================")

SV     CAF        Ca       Cb        Cc         Cd        
1.34 | 10000 | 3874.26 | 1072.44 | 1330.09 | 1861.61
